<a href="https://colab.research.google.com/github/rodrigosarloaf/projeto_DL_23_1/blob/main/gera_patches.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Esse script foi rodado localmente
# Baixei os dados de entrada e depois subi os outputs

# Pacotes
import os
import time
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import h5py
from scipy import stats
import pickle
from PIL import Image, ImageDraw, ImageFont

# Função para gerar patches sem overlap - tamanho fixo de (256,256)
def split_image(dados_in, input_mes_ano, output_path):
    img = Image.fromarray(np.uint8(dados_in))
    width, height = img.size
    patch_size = 256
    n_width = dados_in.shape[0] // patch_size
    n_height = dados_in.shape[1] // patch_size
    count = 0
    for i in range(n_width):
      box = (i * patch_size, 0, (i + 1) * patch_size, height)
      if i == n_width - 1:
        # Last split includes remaining height
        box = (i * patch_size, 0, width, height)
      for j in range(n_height):
        # Adjust box for each row of splits
        row_box = (box[0], j * patch_size, box[2], (j + 1) * patch_size)
        if j == n_height - 1:
          # Last row includes remaining height
          row_box = (box[0], j * patch_size, box[2], height)
        split = img.crop(row_box)
        # Voltando para array
        salvar = np.asarray(split)
        salvar = salvar.astype('float32')
        # Salvando
        candidato = output_path + input_mes_ano + "_patch_" + str(count) + ".pkl"
        if not os.path.exists(candidato):
          with open(candidato, 'wb') as handle:
            pickle.dump(salvar, handle, protocol=pickle.HIGHEST_PROTOCOL)
        if (count % 25) == 0:
          print("count = " + str(count))
        count += 1

# Criando caminhos
path = "semana/"
arqs = []
arqs_title = []
for i in ['2016','2017','2018','2019','2020','2021']:
  for j in range(60):
    candidato = path+i+"_"+str(j)+".pkl"
    if os.path.exists(candidato):
      arqs.append(candidato)
      arqs_title.append(i+"_"+str(j))

print(len(arqs))
print(len(os.listdir(path)))

for i in range(len(arqs)):
  with open(arqs[i], 'rb') as handle:
    dados = pickle.load(handle)
  split_image(dados, arqs_title[i], 'patches/')
  print(arqs_title[i])

len(os.listdir('patches/')) == 100*len(arqs_title)